## 使用desnet201提取特征

训练效果很差,在10%徘徊

In [1]:
import sys
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mping
import torch
import torch.nn as nn
import torchvision.models as models
import torch.utils.data as data
from torchvision import transforms

from data_loader import *
from model import *
from utils import *

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [117]:
class EncoderDensenet201(nn.Module):
    """remove the last conv network"""
    def __init__(self):
        super(EncoderDensenet201, self).__init__()
        densenet = models.densenet201(pretrained=True)
        for param in densenet.parameters():
            param.requires_grad_(False)
        
        modules = list(densenet.children())[:-1]
        self.densenet = nn.Sequential(*modules)
        self.pool = nn.AvgPool2d(7)
        
    def forward(self, images):
        features = self.densenet(images)
        features = self.pool(features)
        features = features.view(features.size(0), -1)
        return features

In [91]:
testLoader = get_loader(transform_train)
image = testLoader.dataset[0][0].unsqueeze(0)

In [118]:
encoder = EncoderDensenet201()
encoder(image).size()

/usr/local/lib/python3.5/dist-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


torch.Size([1, 1920])

In [83]:
class MultiFCClassify(nn.Module):
    def __init__(self, in_features, class_size):
        super(MultiFCClassify, self).__init__()
        self.fc1 = nn.Linear(in_features, in_features//3)
#         self.bn = nn.BatchNorm1d(in_features//3)
#         self.drop = nn.Dropout(0.3)
        self.fc2 = nn.Linear(in_features//3, class_size)
    
    def forward(self, features):
        y = self.fc1(features)
#         y = self.bn(y)
#         y = self.drop(y)
        y = self.fc2(y)
        return y

In [69]:
def valid_class_acc(classify_model, valid_data_loader, device):
    classify_model = classify_model.eval()
    
    indices = valid_data_loader.dataset.get_train_indices()
    new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
    valid_data_loader.batch_sampler.sampler = new_sampler
    
    embeds, targets = next(iter(valid_data_loader))
    
    embeds = embeds.squeeze(1)
    targets = targets.type(torch.LongTensor).to(device)
        
    outputs = classify_model(embeds)
    
    predict_result = outputs.argmax(1)
    size = len(predict_result)
    accuracy = torch.sum(predict_result == targets).item() / size * 100
    
    return accuracy

### set high parameters

In [119]:
batch_size = 256
num_epochs = 100
extract_size = 1920
class_size = 61


# 图片格式转化
transform_train = transforms.Compose([
    transforms.Resize([224,224]),
#     transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
transform_vaild = transforms.Compose([
    transforms.Resize([224,224]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

### load encoded datas

In [120]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# exract the images to embedding tensor
# remove the last conv layer
encoder = EncoderDensenet201()
encoder = encoder.to(device)

# train_fold = './bottle_neck/densenet201_train'
# data_loader = get_encoder_loader_fold(transform_train, encoder, device, train_fold, load=True, mode='train', batch_size=batch_size)

print("\n")
valid_flod = './bottle_neck/densenet201_valid'
valid_data_loader = get_encoder_loader_fold(transform_vaild, encoder, device, valid_flod, load=True, mode='valid', batch_size=batch_size)

/usr/local/lib/python3.5/dist-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)




encoding 4981/4982->99.98%, spent_time:6:38.34

### init model

使用两层网络,结果过拟合,训练准确率很高,验证结果比较差,使用再试

In [123]:
data_loader = valid_data_loader

In [124]:
# set the total number of training steps per epoch
total_step = int(len(data_loader.dataset)/batch_size)

classify_model = MultiFCClassify(extract_size, class_size)
classify_model = classify_model.to(device)

criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# with RMSprop to slow the desent gradient progress
optimizer = torch.optim.Adam(classify_model.parameters(), lr=0.001)

### Load the best trained model,yet!

In [8]:
# classify_model.load_state_dict(torch.load('./models/class_single_resnet18rm1layer_last.pkl'))

### time to train model

In [57]:
optimizer = torch.optim.SGD(classify_model.parameters(), lr=0.0001)

In [125]:
best_acc = 0


for epoch in range(1, num_epochs+1):
    start = time.time()
    classify_model = classify_model.train()
    
    for i_step in range(1, total_step+1):
        
        # Ramdomly get samples
        indices = data_loader.dataset.get_train_indices()
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        embeds, targets = next(iter(data_loader))
        
        embeds = embeds.squeeze(1)
        targets = targets.type(torch.LongTensor).to(device)
        
        classify_model.zero_grad()
        
        outputs = classify_model(embeds)
        
        loss = criterion(outputs, targets.view(-1))
        
        loss.backward()
        
        optimizer.step()
        
        if i_step%20 == 0:
            # calculate the status
            predict_result = outputs.argmax(1)
            accuracy = torch.sum(predict_result == targets).item() / batch_size * 100
            best_acc = accuracy if accuracy > best_acc else best_acc
            
            stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Accuracy: %.2f%%, Best_acc: %.2f%%\
            ' % (epoch, num_epochs, i_step, total_step, loss.item(), accuracy, best_acc)
            print('\r' + stats, end='')
            sys.stdout.flush()
    valid_acc = valid_class_acc(classify_model, valid_data_loader, device)
    print('\n Epoch {}, spent time:{:.2f}s, valid: {:.2f}%'.format(epoch, time.time()-start, valid_acc))       
    if epoch%10 == 0:
        torch.save(classify_model.state_dict(), os.path.join('./models', 'class_single_densenet121_%d.pkl' % epoch))
torch.save(classify_model.state_dict(), os.path.join('./models', 'class_single_densenet121_last.pkl'))


 Epoch 1, spent time:1.81s, valid: 7.42%

 Epoch 2, spent time:1.77s, valid: 6.64%

 Epoch 3, spent time:1.78s, valid: 10.55%

 Epoch 4, spent time:1.74s, valid: 8.20%

 Epoch 5, spent time:1.74s, valid: 8.98%

 Epoch 6, spent time:1.72s, valid: 7.81%

 Epoch 7, spent time:1.72s, valid: 9.38%

 Epoch 8, spent time:1.70s, valid: 5.86%

 Epoch 9, spent time:1.72s, valid: 8.98%

 Epoch 10, spent time:1.72s, valid: 7.03%

 Epoch 11, spent time:1.72s, valid: 5.08%

 Epoch 12, spent time:1.73s, valid: 3.52%

 Epoch 13, spent time:1.75s, valid: 7.81%

 Epoch 14, spent time:1.74s, valid: 5.47%

 Epoch 15, spent time:1.80s, valid: 8.20%

 Epoch 16, spent time:1.75s, valid: 3.91%

 Epoch 17, spent time:1.71s, valid: 4.69%

 Epoch 18, spent time:1.72s, valid: 7.03%

 Epoch 19, spent time:1.76s, valid: 3.52%

 Epoch 20, spent time:1.74s, valid: 7.03%

 Epoch 21, spent time:1.74s, valid: 8.98%

 Epoch 22, spent time:1.73s, valid: 5.86%

 Epoch 23, spent time:1.72s, valid: 5.47%

 Epoch 24, spent t

### How good is the model?

In [53]:
classify_model.load_state_dict(torch.load('./models/class_single_densenet121_last.pkl'))

In [62]:
classify_model = classify_model.eval()
predict = []
total = len(valid_data_loader.dataset)
for idx in range(total):
    embed, target = valid_data_loader.dataset[idx]
    p = classify_model(embed).argmax().item()
    predict.append(p == target)
    
accuracy = sum(predict)/len(predict)
print('The final accuracy is %.2f%%.' % (accuracy*100))

The final accuracy is 69.43%.


## Why? Why the random validation accuracy is good, but not the whole dataset?

## With 100 epoch the result is the best,yet.

In [23]:
!mv ./models/class_multify_rm1layer_100.pkl ./models/class_multify_rm1layer_good.pkl